<a href="https://colab.research.google.com/github/bigdata210/Python/blob/main/Python%20basis/Simple_Linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
PATH_DATA = "/content/drive/MyDrive/2023/kaggle/AMP®-Parkinson's Disease Progression Prediction/"
print(PATH_DATA)

/content/drive/MyDrive/2023/kaggle/AMP®-Parkinson's Disease Progression Prediction/


In [ ]:
# 라이브러리 활성화
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Gradient Boosting Model 
import lightgbm as lgb
# 시간이 걸리는 작업의 경우 상태 확인
import tqdm
# 정규 표현의 패턴을 이용한 문자열 추출, 치환, 분할 가능
import re
# 반복되는 데이터 처리 기능 / 순열
from itertools import product
# 모듈 고차 함수 사용시 이용
from functools import reduce

# 상황에 다라 경고 메세지 사용
import warnings


warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)

pd.set_option('display.float_format', lambda x:'%.3f' %x)

## 데이터 불러오기



In [ ]:
target_data = pd.read_csv(PATH_DATA + 'train_clinical_data.csv')
sub_target = pd.read_csv(PATH_DATA + 'supplemental_clinical_data.csv')
peptides_data = pd.read_csv(PATH_DATA + 'train_peptides.csv')
proteins_data = pd.read_csv(PATH_DATA + 'train_proteins.csv')

# 데이터 컬럼 확인
target_data.shape, peptides_data.shape, proteins_data.shape

((2615, 8), (981834, 6), (232741, 5))

In [ ]:
# target 데이터 합치기
target_data = pd.concat([target_data, sub_target], axis=0 ).reset_index(drop=True)

# visit_month에서 5월 제외 후 복사
target_data = target_data[target_data.visit_month !=5].copy()

# 데이터 속성 이해
target_data.shape, target_data.visit_id.nunique(), target_data.patient_id.nunique(), target_data.visit_month.nunique()

((4720, 8), 4720, 1019, 17)

In [ ]:
# upd23b_clinical_state_on_medication의 빈 값을 unknown 변경.
target_data.upd23b_clinical_state_on_medication.fillna('unknown', inplace =True)

In [ ]:
# na 갯수 확인
target_data.isna().sum()

## upd23b_clinical_state_on_medication에서 na가 없다는 것을 확인할 수 있음.

visit_id                                  0
patient_id                                0
visit_month                               0
updrs_1                                  96
updrs_2                                  98
updrs_3                                  30
updrs_4                                1863
upd23b_clinical_state_on_medication       0
dtype: int64

In [ ]:
# 묶어줄 컬럼 리스트로 저장
id_cols =['visit_id', 'patient_id', 'visit_month']
target_cols = ['updrs_1', 'updrs_2', 'updrs_3', 'updrs_4']
month_list = [0, 6, 12, 24]

- test data에서는 약물을 사용했는지 여부를 알 수 없으므로 약물 사용여부 알 수 없는 것으로 채워서 영향을 확인해야 함!


In [ ]:
def create_target_features(target_data, test=None):
  if test is None:
  # test가 없으면, test1을 생성하고,
    test1 = target_data[['patient_id','visit_month','upd23b_clinical_state_on_medication']].copy()
  # test1에 'upd23b_clinical_state_on_medication'의 값을 unknown로 채워라.
    test1['upd23b_clinical_state_on_medication'] = test1['upd23b_clinical_state_on_medication'].fillna('unknown')
  # test가 있으면 test1에 복사하고,
  else:
    test1 = test.copy()
    # 컬럼 안에 속해있지 않으면, 'upd23b_clinical_state_on_medication'를 unknown로 채워라.
    if 'upd23b_clinical_state_on_medication' not in test1.columns.tolist():
     test1['upd23b_clinical_state_on_medication'] = 'unknown'
    # 컬럼 안에 속해 있으면 unknown으로 채워넣어라.
    else:
     test1['upd23b_clinical_state_on_medication'] =test1['upd23b_clinical_state_on_medication'].fillna('unknown')
    # 중복데이터를 제거하라.
    test1 = test1[['patient_id', 'visit_month', 'upd23b_clinical_state_on_medication']].drop_duplicates().copy()
    
    patients = test1.patient_id.unique()
    visit_months=test1.visit_month.unique()

    test1['visit_month_orig'] = test1['visit_month']

    test_data = pd.DataFrame()
    for patient in patients:
      for month in [0,6,12,24]:
         p_data = test1[test1['patient_id'] ==patient].copy()
         p.data['visit_month'] = p_data['visit_month'].values + month
         test_data = pd.concat([test_data,p_data], axis=0)
      test1 = test_data.copy()

    # 결측값을 방문_월 및 약물별 중앙값으로 바꾸기
    target_data1 = target_data.copy()
    tmeds = target_data1.groupby(['visit_month','upd23b_clinical_state_on_medication'],group_keys =False)[target_cols].apply(lambda x: x.fillna(x.median())).sort_index()
    target_data1 = target_data1[['patient_id','visit_month','upd23b_clinical_state_on_medication']].join(tmeds)

    # 그룹 기능 구축
    all_grp_cols = [['visit_month'],['upd23b_clinical_state_on_medication']] + [['visit_month','upd23b_clinical_state_on_medication']]
    
    # 중복값 제거
    target_data2 = target_data1[['visit_month','upd23b_clinical_state_on_medication']].drop_duplicates()
    for grp_col in all_grp_cols:    
        temp = target_data1.groupby(grp_col)[target_cols].agg(['min','max','mean','median','sum','std'])
        temp.columns = [i+'_' + j + '_'+ '_'.join(grp_col) for i,j in temp.columns]
        target_data2 = target_data2.join(temp,on = grp_col)
        target_data2  = target_data2.join(target_data1.groupby(grp_col)['updrs_1'].count().rename('_'.join(grp_col)+'_count'),
                                          on = grp_col)

    # 약물 복용 플래그
    # 각 컬럼에서 값이 각각 'unknown','Off','On'인 경우 1, 그렇지 않은 경우 0으로 간주
    test1['med_unknown'] = (test1['upd23b_clinical_state_on_medication'] == 'unknown').astype(int)
    test1['med_off'] = (test1['upd23b_clinical_state_on_medication'] == 'Off').astype(int)
    test1['med_on'] = (test1['upd23b_clinical_state_on_medication'] == 'On').astype(int)
    
    model_data = test1.merge(target_data2,on = ['visit_month','upd23b_clinical_state_on_medication'],how = 'inner').drop(columns = 'upd23b_clinical_state_on_medication')
    # feature_cols에서 컬럼(환자ID 제외)후 리스트로 넣어라.
    feature_cols = model_data.drop(columns = ['patient_id']).columns.tolist()


In [ ]:
target_data

,visit_id,patient_id,visit_month,updrs_1,updrs_2,updrs_3,updrs_4,upd23b_clinical_state_on_medication
0,55_0,55,0,10.000,6.000,15.000,NaN,unknown
1,55_3,55,3,10.000,7.000,25.000,NaN,unknown
2,55_6,55,6,8.000,10.000,34.000,NaN,unknown
3,55_9,55,9,8.000,9.000,30.000,0.000,On
4,55_12,55,12,10.000,10.000,41.000,0.000,On
...,...,...,...,...,...,...,...,...
4832,65303_36,65303,36,4.000,1.000,26.000,0.000,unknown
4833,65382_0,65382,0,NaN,NaN,0.000,NaN,unknown
4834,65405_0,65405,0,5.000,16.000,31.000,0.000,unknown
4836,65530_0,65530,0,10.000,6.000,24.000,0.000,unknown


In [ ]:
create_target_features(target_data)

In [ ]:
# 기능 적용
model_data, feature_cols = create_target_features(target_data)
model_data.shape, len(feature_cols)

NameError: ignored

In [ ]:
# model 데이터에서 patient_id, visit_month의 중복 제거값을 제거하고 형태 확인 
model[['patient_id','visit_month']].drop_duplicates().shape, model.shape

# ((4720, 2), (4720, 84)) 
# 2행의 중복값이 있음.

In [ ]:
# 평가 지표
def smape(y_true, y_pred):
    y_true = np.expm1(y_true) +1
    y_pred = np.expm1(y_pred) +1
    
    return 100/len(y_true) * np.sum(2* np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

# y_true : e^y_true +1
# y_pred : e^y_pred +1

In [ ]:
# 모델링할 라이브러리 활성화
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler

from sklearn.model_selection import GroupKFold, GroupShuffleSplit
from sklearn.svm import SVR, LinearSVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet, LinearRegression, PoissonRegreesion
from sklearn.model_selection import cross_validata, cross_val_predict
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import BaggingRegressor, GradientBoostingRegressor, StackingRegressor
import lightgbm as lgb

In [ ]:
# 교차 유효성 검사를 위한 점수확인
smape_ = make_scorer(smape,greater_is_better=False)

In [ ]:
def train_linear_model(model, feature_cols):
                  # 회귀분석
    lgb_params = { 'objective' : 'regression',
                  # 각 Tree에 영향을 미치는 변수 
                 'learning_rate': 0.01,
                  # Tree의 최대 깊이
                 'max_depth': 3,
                  # 전체 Tree의 leave 수
                 'num_leaves':5,
                  # 최종 리프 노드가 되기 위한 레코드 수
                 'min_child_samples':25,
                  # L1 정규화 적용 값 (클수록 과적합 감소)
                 'reg_alpha':25,
                  # L2 정규화 적용 값 (클수록 과적합 감속)
                 'reg_lambda':25,
                  # 반복 수행 트리 개수 지정
                 'n_estimators':100,
                  # 개별트리 학습 시 선택되는 피처 비율
                 'colsample_bytree':0.5,
                  # 데이터 샘플링 비율
                  'subsample': 0.5,
                  # 난수의 시드값을 지정해주는 매개변수 (재현성 보장)
                  'random_state':2023}

# MinMaxscaler를 사용하여 열 크기 조정하기
      # model_data_x : 독립변수를 담은 DataFrame
      model_data_x = model_data.copy()
      models = {}

      gkf = GroupKFold(n_splits = 5)

      score_trn = []; score_test = []

      for target in target_cols:

        # updrs_3의 값이 있다면,
        # updrs_3을 찾은 인덱스를 반환하고, 찾지못하면 -1을 반환
        if target.find('updrs_3')>=0:
          # target의 0의 모든 항목을 pd에서 누락되거나 null값을 나타내는 pd.NA로 바꿈.
          y = model_data[target].replace(0,pd.NA).copy()
          # 누락된 값 NaN 이나 None이 있는 모든 행을 제거하고 복사
          y = model_data[target].dropna().copy()
            
        else:
            y = model_data[target].dropna().copy()
        # np.log1p() : 입력값에 1을 더한 후 자연 로그 계산
                     #( undefined error를 피하기 위해 1을 더해줌)
        y = y.apply(np.log1p).copy()
        # y : 종속변수 데이터를 담은 Series
        # y의 값을 리스트로 반환하고 model_data_x의 인덱스와 동일
        # X는 feature_cols 리스트에 해당하는 독립변수 데이터를 담은 데이터 프레임
        X = model_data_x.loc[y.index.tolist(), feature_cols]
        # model_data_x에서 'patient_id' 컬럼을 선택 후 y의 인덱스에 해당하는 행을 추출 새로운 Series
        # 같은 환자가 여러 행에 나타날 수 있으므로 환자 구분시 사용
        groups = model_data_x.loc[y.index.tolist(), 'patient_id']
        
        # 각 모델은 기능 확장 및 회귀라는 두 가지 처리 단계로 구성된 파이프 라인
        # 강력한 scaler가 포함된 선형 지원 벡터 회귀 모델
        model1 = Pipeline([('scaler', RobustScaler()),
                           ('lsvr', LinearSVR(random_state=2023))])
        
        # MinMaxscaler가 포함된 푸아송 회귀 모델
        model2 = Pipeline([('scaler', MinMaxScaler()),
                           ('poisson', PoissonRegressor())])
        
        # 방사형 기저 함수(RBF), scaler가 있는 서포트 벡터 회귀 모델 
        model3 = Pipeline([('scaler', RobustScaler()),
                           ('svr_rbf', SVR())])
        
        # 표준 scaler가 있는 선형 회귀 모델
        # 평균을 제거하고 단위 분산으로 scaler하여 입력 특징을 표준화하는데 사용
        model4 = Pipeline([('scaler', StandardScaler()),
                           ('lr', LinearRegression())])
        
        model5 = lgb.LGBMRegressor(**lgb_params)                   

